In [4]:
using DecisionTree
using Random
using Statistics: mean
using CategoricalArrays

# Define the AdaBoost algorithm
function adaboost(X, y, T)
    n, m = size(X)
    
    # Initialize weights
    weights = ones(n) / n
    
    # Storage for weak classifiers, their weights, and alpha values
    weak_classifiers = Vector{typeof(DecisionTreeClassifier)}(undef, T)
    classifier_weights = zeros(T)
    alphas = zeros(T)
    
    for t in 1:T
        # Train a weak learner (decision stump in this case)
        weak_classifier = DecisionTreeClassifier(max_depth=1)
        fit!(weak_classifier, X, categorical(y), sample_weights=weights)
        
        # Make predictions
        y_pred = DecisionTree.predict(weak_classifier, X)
        
        # Calculate error
        err = sum(weights .* (y .!= y_pred))
        
        # Avoid division by zero
        err = max(err, eps())
        
        # Calculate classifier weight and alpha
        beta = err / (1 - err)
        alpha = log(1 / beta)
        
        # Update weights
        weights *= exp(alpha * (y .!= y_pred))
        
        # Normalize weights
        weights /= sum(weights)
        
        # Store weak classifier, its weight, and alpha
        weak_classifiers[t] = weak_classifier
        classifier_weights[t] = alpha
        alphas[t] = alpha
    end
    
    return weak_classifiers, classifier_weights, alphas
end

# # Rest of the code remains the same

# # Test the AdaBoost model on a new example
# new_example = [7.0 8.0]
# prediction = adaboost(weak_classifiers, classifier_weights, alphas, new_example)

# println("AdaBoost Prediction for $new_example: ", prediction)


adaboost (generic function with 1 method)

In [5]:
# Example usage
# Generate synthetic data
Random.seed!(123)
X = randn(100, 2)
y = rand([-1, 1], 100)

# Train AdaBoost
weak_classifiers, classifier_weights, alphas = adaboost(X, y, 50)

# Test the AdaBoost model on a new example
new_example = [0.5, 1.0]
prediction = adaboost(weak_classifiers, classifier_weights, alphas, new_example)

println("AdaBoost Prediction for $new_example: ", prediction)

LoadError: MethodError: no method matching fit!(::DecisionTreeClassifier, ::Matrix{Float64}, ::CategoricalVector{Int64, UInt32, Int64, CategoricalValue{Int64, UInt32}, Union{}}; sample_weights::Vector{Float64})

[0mClosest candidates are:
[0m  fit!(::DecisionTreeClassifier, ::Any, ::Any)[91m got unsupported keyword argument "sample_weights"[39m
[0m[90m   @[39m [36mDecisionTree[39m [90m~/.julia/packages/DecisionTree/0Dw1P/src/[39m[90m[4mscikitlearnAPI.jl:91[24m[39m
[0m  fit!([91m::AdaBoostStumpClassifier[39m, ::Any, ::Any)[91m got unsupported keyword argument "sample_weights"[39m
[0m[90m   @[39m [36mDecisionTree[39m [90m~/.julia/packages/DecisionTree/0Dw1P/src/[39m[90m[4mscikitlearnAPI.jl:518[24m[39m
[0m  fit!([91m::DecisionTreeRegressor[39m, ::AbstractMatrix, ::AbstractVector)[91m got unsupported keyword argument "sample_weights"[39m
[0m[90m   @[39m [36mDecisionTree[39m [90m~/.julia/packages/DecisionTree/0Dw1P/src/[39m[90m[4mscikitlearnAPI.j

# Reference
- [ ] [AdaBoost, Clearly Explained](https://www.youtube.com/watch?v=LsK-xG1cLYA)
- [ ] [[05x06] DecisionTree.jl: Decision Tree, Random Forest, AdaBoost | Julia Supervised Machine Learning](https://www.youtube.com/watch?v=XTApO31m3Xs)